In [1]:
import numpy as np
import torch

from tqdm.auto import tqdm
from pathlib import Path

import itertools

import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), verbose=True)

True

In [13]:
!rm -r /home/huze/.cache/checkpoints

E0311 07:33:30.520751513 3249277 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [3]:
exp_configs = sorted([p for p in Path('../../src/config/experiments/algonauts2021/').iterdir() if p.name.endswith('.yml')])[::-1]

In [4]:
exp_configs

[PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_audio_vgg.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_swin.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_resnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_3d_flow.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_simclr.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_segswin.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_pyconvsegnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_mobyswin.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_densnet.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_colorizer.yml'),
 PosixPath('../../src/config/experiments/algonauts2021/algonauts2021_2d_bdcnvgg.yml')]

In [5]:
os.environ['RESULTS_DIR']

'/data/huze/ray_results/algonauts2021/'

In [6]:
import pandas as pd
defrost_df = pd.read_csv('./config/defrost_df.csv')

In [7]:
defrost_df

,Unnamed: 0,MODEL.BACKBONE.NAME,DATASET.ROI,score,defrost_score
0,34,2d_bdcnvgg_warp_3d,WB,0.038811,0.018
1,37,2d_colorizer_warp_3d,WB,0.088229,0.045
2,54,2d_simclr_warp_3d,WB,0.135406,0.069
3,55,i3d_flow,WB,0.124815,0.063
4,89,i3d_rgb,WB,0.136212,0.069
5,90,3d_swin,WB,0.140836,0.069
6,123,audio_vggish,WB,0.058880,0.030
7,124,2d_pyconvsegnet_warp_3d,WB,0.134461,0.066
8,125,2d_densnet_warp_3d,WB,0.118351,0.060
9,144,2d_seg_swin_warp_3d,WB,0.115133,0.057


In [8]:
from src.utils.runs import my_query_df
from src.config.config import convert_to_dict, flatten

In [9]:
from src.grid_runner import run_single_train
from src.grid_runner import run_single_tune_config
from src.config import get_cfg_defaults, combine_cfgs
from ray import tune

In [10]:
debug = False

In [11]:
def get_defrost_score(defrost_df, cfg):
    cfg_dict = flatten(convert_to_dict(cfg))
    defrost_score = my_query_df(defrost_df, equal_dict={k: cfg_dict[k] for k in ['MODEL.BACKBONE.NAME', 'DATASET.ROI']}).defrost_score.item()
    return defrost_score

In [17]:
analysis_list = []
for exp_config in exp_configs:
    cfg = combine_cfgs(
        path_cfg_data=exp_config,
        list_cfg_override=['DEBUG', debug]
    )
    
    name = exp_config.name.replace('.yml', '')
    
    # defrost_score = get_defrost_score(defrost_df, cfg)
    model_defrost_df = defrost_df[defrost_df['MODEL.BACKBONE.NAME'] == cfg.MODEL.BACKBONE.NAME]
    
    # single-layer 4x4 run
    analysis = tune.run(
        tune.with_parameters(
            run_single_tune_config,
            cfg=cfg
        ),
        config={
            'DATASET.ROI': tune.grid_search(['WB']),
            'MODEL.BACKBONE.LAYERS': tune.grid_search([[i] for i in cfg.MODEL.BACKBONE.LAYERS]),
            'MODEL.NECK.SPP_LEVELS': tune.grid_search([[i] for i in cfg.MODEL.NECK.SPP_LEVELS]),
            'MODEL.NECK.FIRST_CONV_SIZE': tune.sample_from(
                lambda spec: {1: 2048, 2: 1024, 3: 512, 6: 256, 7: 256, 14: -1, 28: -1, 42: -1, 56: -1}[
                    np.max(spec.config['MODEL.NECK.SPP_LEVELS'])]),
            'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': tune.sample_from(
                lambda spec: model_defrost_df[model_defrost_df['DATASET.ROI'] == spec.config['DATASET.ROI']]['defrost_score'].item())
            # tune.grid_search([defrost_score]),
        },
        local_dir=cfg.RESULTS_DIR,
        resources_per_trial={"cpu": 4, "gpu": 1},
        name=name + '_singlelayer_defrost_WB',
        verbose=1,
        resume='AUTO',
    )
    analysis_list.append(analysis)
    
    # multi-layer 1 run
    analysis = tune.run(
        tune.with_parameters(
            run_single_tune_config,
            cfg=cfg
        ),
        config={
            'DATASET.ROI': tune.grid_search(['WB']),
            'MODEL.BACKBONE.LAYERS': tune.grid_search([cfg.MODEL.BACKBONE.LAYERS]),
            'MODEL.NECK.SPP_LEVELS': tune.grid_search([cfg.MODEL.NECK.SPP_LEVELS]),
            'MODEL.NECK.FIRST_CONV_SIZE': tune.sample_from(
                lambda spec: {1: 2048, 2: 1024, 3: 512, 6: 256, 7: 256, 14: -1, 28: -1, 42: -1, 56: -1}[
                    np.max(spec.config['MODEL.NECK.SPP_LEVELS'])]),
            'TRAINER.CALLBACKS.BACKBONE.DEFROST_SCORE': tune.sample_from(
                lambda spec: model_defrost_df[model_defrost_df['DATASET.ROI'] == spec.config['DATASET.ROI']]['defrost_score'].item())
            # tune.grid_search([defrost_score]),
        },
        local_dir=cfg.RESULTS_DIR,
        resources_per_trial={"cpu": 4, "gpu": 1},
        name=name + '_multilayer_defrost_WB',
        verbose=1,
        resume='AUTO',
    )
    analysis_list.append(analysis)


2022-03-11 09:21:51,783	INFO tune.py:636 -- Total run time: 351.93 seconds (351.82 seconds for the tuning loop).
